In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('..')

In [2]:
from models import LocalNoiseRBM


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
nv = 5
nh = 3
visfeed = tf.placeholder(shape=(None, nv),dtype=tf.float32)

In [4]:
k=10

In [5]:
rbm = LocalNoiseRBM(nv, nh)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
grads = rbm.nll_gradients(visfeed,k, noise_condition=True)

TypeError: compute_visible_probs() got multiple values for argument 'noise_condition'